# Modelos de Predicción de Ventas con diferentes variables

En este notebook vamos a mostrar las nuevas variables que creamos para probar cómo podiamos mejorar el modelo. Nuestra hipotesis inicial es que al proporcionar más datos relacionales a la Cantidad de Ventas, estas nos ayudan a mejorar el modelo. Vamos a probar con dos modelos: Linear Regression y Random Forest. En el notebook se mostrará la evolución de los modelos con las nuevas variables.

In [1]:
# Importamos las librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import hvplot.pandas
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('once')

In [2]:
tienda = pd.read_csv('tienda.csv')
tipos_de_datos = {
    'ID_PRODUCTO': 'object',
    'FECHA_COMPRA': 'datetime64[ns]',
    'CADUCIDAD': 'datetime64[ns]',
    'CANTIDAD': 'float64',
    'PRECIO_TOTAL': 'float64',
    'INDEX_CLIENTES': 'int32',
    'ETIQUETA_CANAL': 'int64',
    'CATEGORIA': 'object',
}
tienda = tienda.astype(tipos_de_datos)
tienda = tienda.loc[tienda['CANTIDAD'] >= 0]
tienda['mes'] = tienda['FECHA_COMPRA'].dt.month
tienda['semana'] = tienda['FECHA_COMPRA'].dt.isocalendar().week
tienda.head()

,ID_PRODUCTO,FECHA_COMPRA,CADUCIDAD,CANTIDAD,PRECIO_TOTAL,INDEX_CLIENTES,ETIQUETA_CANAL,CATEGORIA,mes,semana
0,00026751,2023-07-29,2023-08-01,1.00,8.33,0,1,Salchichon i embutidos,7,30
1,00026751,2023-07-08,2023-10-01,0.59,4.94,5,1,Salchichon i embutidos,7,27
2,00026751,2023-07-08,2023-10-01,0.35,2.93,5,1,Salchichon i embutidos,7,27
3,00026751,2023-07-08,2023-10-01,0.32,2.64,5,1,Salchichon i embutidos,7,27
4,00026751,2023-07-09,2023-10-01,1.20,10.04,5,1,Salchichon i embutidos,7,27


### Nuevas Variables

In [3]:
# Organizamos las fechas: 
tienda = tienda.sort_values(by=['ID_PRODUCTO', 'FECHA_COMPRA'])
fecha_min_global = tienda['FECHA_COMPRA'].min()
all_combinations = pd.DataFrame()
for product_id, group in tienda.groupby('ID_PRODUCTO'):
    start_date = fecha_min_global
    end_date = start_date + pd.DateOffset(weeks=21)
    date_range = pd.date_range(start=start_date, end=end_date, freq='W-Mon')
    product_combinations_df = pd.DataFrame({'ID_PRODUCTO': product_id, 'FECHA_COMPRA': date_range})
    all_combinations = pd.concat([all_combinations, product_combinations_df])
tienda_organizada = pd.merge(all_combinations, tienda, on=['ID_PRODUCTO', 'FECHA_COMPRA'], how='left')
tienda_organizada['semana'] = tienda_organizada['FECHA_COMPRA'].dt.isocalendar().week
tienda_organizada['mes'] = tienda_organizada['FECHA_COMPRA'].dt.month
tienda_organizada['CANTIDAD'] = tienda_organizada['CANTIDAD'].fillna(0)


In [4]:
tienda_organizada.head()

,ID_PRODUCTO,FECHA_COMPRA,CADUCIDAD,CANTIDAD,PRECIO_TOTAL,INDEX_CLIENTES,ETIQUETA_CANAL,CATEGORIA,mes,semana
0,00000012,2023-07-10,NaT,0.0,NaN,NaN,NaN,NaN,7,28
1,00000012,2023-07-17,NaT,0.0,NaN,NaN,NaN,NaN,7,29
2,00000012,2023-07-24,2026-01-31,1.0,3.61,5.0,2.0,Verduras conservadas,7,30
3,00000012,2023-07-31,2025-01-03,1.0,2.38,43.0,2.0,Verduras conservadas,7,31
4,00000012,2023-08-07,NaT,0.0,NaN,NaN,NaN,NaN,8,32


#### Variables 1: aportan comparación de la cantidad de ventas 
- Número de la semana
- Número del mes
- Suma de cantidad de ventas de dos y tres semanas: Se calculan las sumas acumulativas de las cantidades para las últimas 2 y 3 semanas respectivamente, agrupadas por 'ID_PRODUCTO'.
- Diferencia de ventas entre una, dos y tres semanas. Se calculan las diferencias entre las cantidades de semanas consecutivas, de dos semanas atrás y de tres semanas atrás, agrupadas por 'ID_PRODUCTO'.
- Ventas próxima semana. Se agrega una columna que representa las ventas para la próxima semana al desplazar hacia arriba las cantidades en una unidad, agrupadas por 'ID_PRODUCTO'.

#### Variables 2: que aportan nuevos datos estadisticos de cada semana

#### Dataframe final con Variables

In [5]:
tienda_organizada['semana'] = tienda_organizada['FECHA_COMPRA'].dt.isocalendar().week
tienda_organizada['mes'] = tienda_organizada['FECHA_COMPRA'].dt.month
Variables1 = tienda_organizada.groupby(['ID_PRODUCTO', tienda_organizada['FECHA_COMPRA'].dt.to_period('W-Mon')])['CANTIDAD'].sum().reset_index()
Variables1['semana'] = tienda_organizada['FECHA_COMPRA'].dt.isocalendar().week
Variables1['suma_2_semanas'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].rolling(window=2, min_periods=1).sum().reset_index(level=0, drop=True)
Variables1['suma_3_semanas'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].rolling(window=3, min_periods=1).sum().reset_index(level=0, drop=True)
Variables1['dif_una_semana'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].diff()
Variables1['dif_dos_semanas'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].diff(periods=2)
Variables1['dif_tres_semanas'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].diff(periods=3)
Variables1['ventas_proxima_semana'] = Variables1.groupby('ID_PRODUCTO')['CANTIDAD'].shift(-1)
Variables1 = Variables1.dropna()

Variables2 = tienda_organizada.groupby(['ID_PRODUCTO', 'semana']).agg({'CANTIDAD': ['sum', 'max', 'min', 'mean', 'median', 'std', 'skew']}).reset_index()
Variables2.columns = ['ID_PRODUCTO', 'semana', 'CANTIDAD_TOTAL', 'MAX_CANTIDAD', 'MIN_CANTIDAD', 'MEAN', 'MEDIAN', 'ST', 'SKEW']
Variables2 = Variables2.dropna()
Variables2['semana'] = Variables2['semana'].astype(int)

data = pd.merge(Variables1, Variables2, left_on=['ID_PRODUCTO', 'semana'], right_on=['ID_PRODUCTO', 'semana'], how='inner')

In [6]:
data.head()

,ID_PRODUCTO,FECHA_COMPRA,CANTIDAD,semana,suma_2_semanas,suma_3_semanas,dif_una_semana,dif_dos_semanas,dif_tres_semanas,ventas_proxima_semana,CANTIDAD_TOTAL,MAX_CANTIDAD,MIN_CANTIDAD,MEAN,MEDIAN,ST,SKEW
0,00000012,2023-10-10/2023-10-16,3.0,42,3.0,3.0,3.0,3.0,3.0,0.0,3.0,1.0,1.0,1.000000,1.0,0.00000,0.000000
1,00000012,2023-10-17/2023-10-23,0.0,42,3.0,3.0,-3.0,0.0,0.0,0.0,3.0,1.0,1.0,1.000000,1.0,0.00000,0.000000
2,00000012,2023-10-24/2023-10-30,0.0,42,0.0,3.0,0.0,-3.0,0.0,0.0,3.0,1.0,1.0,1.000000,1.0,0.00000,0.000000
3,00000014,2023-08-15/2023-08-21,0.0,31,3.0,3.0,-3.0,0.0,-4.0,8.0,4.0,2.0,1.0,1.333333,1.0,0.57735,1.732051
4,00000014,2023-08-22/2023-08-28,8.0,31,8.0,11.0,8.0,5.0,8.0,0.0,4.0,2.0,1.0,1.333333,1.0,0.57735,1.732051


### Linear Regression con Variables 1

In [7]:
X = data[['suma_2_semanas', 'suma_3_semanas', 'dif_una_semana', 'dif_dos_semanas', 'dif_tres_semanas']]
y = data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 34.726814122937064
R^2 Score: 0.10707086515564235


### Linear Regression con Variables 2

In [8]:
merged_data = pd.merge(Variables1, Variables2, on=['ID_PRODUCTO', 'semana'], how='inner')
X = merged_data[['CANTIDAD_TOTAL', 'MAX_CANTIDAD', 'MIN_CANTIDAD', 'MEAN', 'MEDIAN', 'ST', 'SKEW']]
y = merged_data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 34.5625389124218
R^2 Score: 0.11549888689700638


### Linear Regression con Todas nuevas variables

In [9]:
features = data.drop(['ID_PRODUCTO', 'ventas_proxima_semana', 'FECHA_COMPRA'], axis=1)
target = data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 33.976926184397406
R^2 Score: 0.14521816983747238


### Random Forest con variables 1

In [10]:
X = data[['suma_2_semanas', 'suma_3_semanas', 'dif_una_semana', 'dif_dos_semanas', 'dif_tres_semanas']]
y = data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 33.29127332170581
R^2 Score: 0.17936899531647843


### Random Forest con variables 2

In [11]:
merged_data = pd.merge(Variables1, Variables2, on=['ID_PRODUCTO', 'semana'], how='inner')
X = merged_data[['CANTIDAD_TOTAL', 'MAX_CANTIDAD', 'MIN_CANTIDAD', 'MEAN', 'MEDIAN', 'ST', 'SKEW']]
y = merged_data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 31.01529290849313
R^2 Score: 0.2877394633642488


### Random Forest con todas las variables

In [12]:
features = data.drop(['ID_PRODUCTO', 'ventas_proxima_semana', 'FECHA_COMPRA'], axis=1)
target = data['ventas_proxima_semana']
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R^2 Score: {r2}')

Root Mean Squared Error (RMSE): 32.18733783748141
R^2 Score: 0.2328907398842346


## Conclusiones
Los resultados finales de todos los modelos da un error muy alto, por lo que las predicciones de ventas no son buenas. Utilizando el modelo de regresión lineal, el que mejor funciona es el que incluye todas las nuevas variables que agregamos. En cambio, en Random Forest el que mejor resultado da es cuando usamos las variables que aportaban datos estadisticos. La escasez de datos no ayudan a obtener un buen resultado, por lo que es importante que la empresa tenga una base de datos más amplia y además tenga otras variables que aporten valor nuevo al análisis. 